In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from konlpy.utils import pprint
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

In [2]:
#크롤링 결과 열어서 포스팅 단위로 불리 저장
doc =open('왕십리역.csv')
data = doc.read()
doc.close()
phrase = data.split('\n')

In [3]:
#포스팅에서 한글 외 문자 제거
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')


In [4]:
#형태소 문장 단위 추출
dic_Twitter = []
for i in range(len(phrase)):
    p = hangul.sub(' ', phrase[i])
    s = Twitter().pos(p, stem=True, norm=True)  ##norm, stem 옵션 추가
    dic_Twitter.append(s)
print(dic_Twitter[:3])

[[('중경', 'Noun'), ('통법', 'Noun'), ('으로', 'Josa'), ('지치다', 'Verb'), ('심신', 'Noun'), ('을', 'Josa'), ('지다', 'Verb'), ('커플', 'Noun'), ('양말', 'Noun'), ('로', 'Noun'), ('달래', 'Noun'), ('보다', 'Verb'), ('왕십리역', 'Noun'), ('레드아이', 'Noun'), ('커플', 'Noun'), ('양말', 'Noun'), ('우정', 'Noun'), ('스타', 'Noun'), ('그램', 'Noun'), ('친', 'Noun'), ('스타', 'Noun'), ('그램', 'Noun')], [('왕꼬치', 'Noun'), ('왕십리역', 'Noun'), ('왕십리', 'Noun'), ('백번', 'Noun'), ('한양대', 'Noun'), ('좋다', 'Adjective'), ('먹다', 'Verb'), ('스타', 'Noun'), ('그램', 'Noun'), ('팔로우', 'Noun'), ('선팔', 'Noun'), ('맞팔', 'Noun'), ('선팔', 'Noun'), ('환영', 'Noun')], [('왕십리', 'Noun'), ('하', 'Noun'), ('왕십리', 'Noun'), ('왕십리역', 'Noun'), ('노블', 'Noun'), ('헤어', 'Noun'), ('소영이', 'Noun'), ('머리', 'Noun'), ('클리', 'Noun'), ('닉', 'Noun'), ('받다', 'Verb'), ('가다', 'Verb'), ('네일', 'Noun'), ('샵', 'Suffix'), ('이', 'Josa'), ('있다', 'Adjective'), ('냉큼', 'Noun'), ('네일', 'Noun'), ('바꾸다', 'Verb'), ('버리다', 'Verb'), ('보통', 'Noun'), ('은', 'Josa'), ('미리', 'Noun'), ('네일', 'Noun'), ('을', 'Josa'

In [5]:
#형용사와 명사 단어만 문장별로 추출
dic_ad_noun=[]
for twit in dic_Twitter:
    c = [x[0] for x in twit if x[1] =="Adjective" or x[1] == "Noun"]
    dic_ad_noun.append(c)
print(dic_ad_noun[:3])

[['중경', '통법', '심신', '커플', '양말', '로', '달래', '왕십리역', '레드아이', '커플', '양말', '우정', '스타', '그램', '친', '스타', '그램'], ['왕꼬치', '왕십리역', '왕십리', '백번', '한양대', '좋다', '스타', '그램', '팔로우', '선팔', '맞팔', '선팔', '환영'], ['왕십리', '하', '왕십리', '왕십리역', '노블', '헤어', '소영이', '머리', '클리', '닉', '네일', '있다', '냉큼', '네일', '보통', '미리', '네일', '준비', '급하다', '일판', '있다', '초', '네일', '미술', '전공', '언니', '슥슥', '그냥', '주심', '짱']]


# word2vec으로 단어과 가까운 단어 출력해보기

In [6]:
#word2vec으로 단어를 공간에 뿌리기 
#size는 차원의 크기, 결과를 보고 결정하자
w2v = models.Word2Vec(dic_ad_noun, size=100)

# K means로 클러스터링 해보기

In [7]:
#K means 시각화
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
rc('font', family=font_name)
flist = matplotlib.font_manager.get_fontconfig_fonts()

C:\Anaconda3\lib\site-packages\matplotlib\font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [8]:
#K means를 위해  TSNE로 다차원상에 있는 데이터를 평면에 흩어지게 표현
from sklearn.manifold import TSNE
tsne = TSNE()
textplot = tsne.fit_transform(w2v[w2v.index2word])

In [9]:
from sklearn import metrics
from sklearn.cluster import KMeans

In [10]:
#K를 10으로 해서 K means 클러스터링한다
kmeans = KMeans(n_clusters=10)
kmeans.fit(textplot)
cluster = kmeans.predict(textplot)

In [11]:
#결과 png로 출력
xlist = []
ylist = []
for x, y  in textplot:
    xlist.append(x)
    ylist.append(y)
plt.figure(figsize=(150,150))
plt.scatter(xlist,ylist, c=cluster)
rc('font',family=font_name)
for k, v, s in zip(xlist,ylist,w2v.index2word):
    if s=='좋다' or s=='맛있다' or s=='다양하다' or s=='필요하다' or s=='착하다':
        plt.text(k,v,s, fontsize=70, color='r', alpha=0.5)
    else:
        plt.text(k,v,s, fontsize=20, alpha=0.9)
plt.savefig('왕십리역_Kmeans.png')
plt.show()